In [2]:
import os

# os.environ["PYLON_CAMEMU"] = "3"

from pypylon import genicam
from pypylon import pylon
import sys
import platform
# Number of images to be grabbed
countOfImagesToGrab = 10
img = pylon.PylonImage()
# Limits the amount of cameras used for grabbing.
maxCamerasToUse = 2

# The exit code of the sample application
exitCode = 0
try:

    # Get the transport layer factory.
    tlFactory = pylon.TlFactory.GetInstance()
    
    # Get all attached devices and exit application if no device is found.

    devices = tlFactory.EnumerateDevices()
    if len(devices) == 0:
        raise pylon.RuntimeException("No camera present.")
        
    # Create an array of instant cameras for the found devices and avoid exceeding a maximum number of devices.
    cameras = pylon.InstantCameraArray(min(len(devices),maxCamerasToUse))
    
    l = cameras.GetSize()
    
    # Create and attach all Pylon Devices.
    for i, cam in enumerate(cameras):
#         print(devices[i].GetDeviceClass())
#         if devices[i].GetDeviceClass() == 'BaslerGigE':
        cam.Attach(tlFactory.CreateDevice(devices[i]))
        
            # Print the model name of the camera.
        print("Using device", cam.GetDeviceInfo().GetModelName())
    # Starts grabbing for all cameras starting with index 0. The grabbing is started for one camera after the other.
    # However, a hardware trigger setup can be used to cause all cameras to grab images synchronously
    # According to deafault configuration, the cameras are set up for free-running continuous acquisition
    
    cameras.Open()
    cameras.StartGrabbing()
    
    # Grab c_countOfImagesToGrab from the cameras.
    for i in range(countOfImagesToGrab):
        if not cameras.IsGrabbing():
            break
            
        grabResult = cameras.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
        
        # When the cameras in the array are created the camera context value
        # is set to the index of the camera in the array
        # The camera context is attached to each grab result and can be used
        # to determine the camera that produced the grab result.
        cameraContextValue = grabResult.GetCameraContext()
        
        #Print the index and the model name of the camera.
        print("Camera ", cameraContextValue, ":", cameras[cameraContextValue].GetDeviceInfo().GetModelName())
        
        #The image data can be processd
        print("GrabSucceeded: ", grabResult.GrabSucceeded())
        print("SizeX: ", grabResult.GetWidth())
        print("SizeY: ", grabResult.GetHeight())
#         img = grabResult.GetArray()
#         print("Gray value of first pixel: ", img[0, 0])
        img.AttachGrabResultBuffer(grabResult)
        if platform.system() == 'Windows':
            # The JPEG format that is used here supports adjusting the image
            # quality( 100 -> best quality, 0 -> poor quality)
            ipo = pylon.ImagePersistenceOptions()
            quality = 90 - i * 10
            ipo.SetQuality(quality)
            
            filename = "images/save_pypylon_img_%d.jpeg" % quality
            img.Save(pylon.ImageFileFormat_Jpeg, filename, ipo)
            
        else:
            filename = "images/save_pypylon_img_%d.png" % i
            img.Save(pylon.ImageFileFormat_Png, filename)
        
        # In order to make it possible to reuse the grab result for grabbing
        # again, we have to release the image (effectively emptying the image object)
        img.Release()
    cameras.StopGrabbing()
    cameras.Close()
        
except genicam.GenericException as e:
    # Error handling
    print("An exception occurred.", e.GetDescription())
    exitCode = 1
    
# Comment the following two lines to disable waiting on exit
sys.exit(exitCode)


Using device acA2500-14gm
Camera  0 : acA2500-14gm
GrabSucceeded:  True
SizeX:  2592
SizeY:  1944
Camera  0 : acA2500-14gm
GrabSucceeded:  True
SizeX:  2592
SizeY:  1944
Camera  0 : acA2500-14gm
GrabSucceeded:  True
SizeX:  2592
SizeY:  1944
Camera  0 : acA2500-14gm
GrabSucceeded:  True
SizeX:  2592
SizeY:  1944
Camera  0 : acA2500-14gm
GrabSucceeded:  True
SizeX:  2592
SizeY:  1944
Camera  0 : acA2500-14gm
GrabSucceeded:  True
SizeX:  2592
SizeY:  1944
Camera  0 : acA2500-14gm
GrabSucceeded:  True
SizeX:  2592
SizeY:  1944
Camera  0 : acA2500-14gm
GrabSucceeded:  True
SizeX:  2592
SizeY:  1944
Camera  0 : acA2500-14gm
GrabSucceeded:  True
SizeX:  2592
SizeY:  1944
Camera  0 : acA2500-14gm
GrabSucceeded:  True
SizeX:  2592
SizeY:  1944


SystemExit: 0